In [4]:
import cv2 as cv
import imutils
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

In [5]:
def run_avg(image, aWeight):
    global bg
    if bg is None:
        bg = image.astype('float')
        print('background captured')
        return
    cv.accumulateWeighted(image, bg, aWeight)
    #print(bg)

In [6]:
def segment (image, threshold=12):
    global bg
    #cv.imshow('back', np.uint8(bg))    
    #cv.imshow('Handy', image)
    diff = cv.absdiff(np.uint8(bg), image)
    cv.imshow('diff', diff)
    
    blur = cv.GaussianBlur(diff, (3, 3), 0)
    blur = cv.medianBlur(diff, 11)
    
    #thresholded = cv.adaptiveThreshold(blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 5, 12)
    thresholded = cv.threshold(diff, threshold, 255, cv.THRESH_BINARY)[1]
    disc = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
    disc1 = cv.getStructuringElement(cv.MORPH_ELLIPSE, (2, 2))
    thresholded = cv.morphologyEx(thresholded, cv.MORPH_CLOSE, disc, iterations =4)
    thresholded = cv.dilate(thresholded, disc, iterations =1)
    
    (cnts, _) = cv.findContours(thresholded.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    if len(cnts) == 0:
        return
    else:
        segmented = max(cnts, key = cv.contourArea)
        return (thresholded, segmented)


In [7]:
def hist_skin(hist, hsv):
    
    dst = cv.calcBackProject([hsv], [0, 1], hist, [0, 200, 0, 256], 1)
    dst1 = dst.copy()
    disc = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
    cv.filter2D(dst, -1, disc, dst)
    blur = cv.GaussianBlur(dst, (11, 11), 0)
    blur = cv.medianBlur(blur, 15)
    #cv.imshow("Blur", blur)
    ret, thresh = cv.threshold(blur, 0, 255, cv.THRESH_BINARY)#+cv.THRESH_OTSU)
    #thresh = cv.merge((thresh, thresh, thresh))
    
    return thresh

In [8]:
bg = None
if __name__ == '__main__':
    
    aWeight = 0.5
    
    flag = False
    
    camera = cv.VideoCapture(0)
    
    top, right, bottom, left = 10, 200, 250, 440
    
    in_top, in_right, in_bottom, in_left = top+50, right+100, bottom-50, left-100
    
    num_frames = 0
    
    while True:
        (ret, frame) = camera.read()
        
        frame = cv.resize(frame, (640, 480))
        frame = cv.flip(frame, 1)
        
        clone = frame.copy()
        
        (height, width) = frame.shape[:2]
        roi = frame[top:bottom, right:left]
        hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
        
        in_roi = frame[in_top:in_bottom, in_right:in_left]
        
        gray = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
        gray = cv.GaussianBlur(gray, (3, 3), 0)
        gray = cv.medianBlur(gray, 3)
        
        keypress = cv.waitKey(1) & 0xFF
        
        if keypress == ord('c'):
            break
            
        elif keypress == ord('s'):
            flag = True
            print('skin hist captured!')
            hsvCrop = cv.cvtColor(in_roi, cv.COLOR_BGR2HSV)
            cv.imshow("i", hsvCrop)
            hist = cv.calcHist([hsvCrop], [0, 1], None, [200, 256], [0, 200, 0, 256])
            cv.normalize(hist, hist, 0, 255, cv.NORM_MINMAX)
            
        if keypress == 13:
            bg = None
            num_frames = 0
        
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            hand = segment(gray)
            
            if hand is not None:
                (thresholded, segmented) = hand
                
                cv.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                if flag:
                    skin = hist_skin(hist, hsv_roi)
                    #print(thresholded.shape, skin.shape)
                    thresholded = cv.bitwise_or(thresholded, skin)
                
                cv.imshow('Thresholded', thresholded)
                
                
        cv.rectangle(clone, (left, top), (right, bottom), (0, 255, 0), 2)
        cv.rectangle(clone, (in_left, in_top), (in_right, in_bottom), (255, 0, 0), 1)
        
        num_frames += 1
        cv.imshow('Video Feed', clone)
            
cv.destroyAllWindows()
camera.release()

background captured
background captured
background captured
background captured
background captured
background captured
background captured
background captured
background captured
background captured
background captured
background captured
background captured
